In [20]:
using RigidBodyDynamics
using StaticArrays
using Parameters
using RobotDynamics
using Rotations
using LinearAlgebra
using ForwardDiff

In [21]:
# Defining Arthur model using RigidBodyDynamics
struct Arthur{T} <: AbstractModel
    mechanism::T
    function Arthur(mechanism)
        T = eltype(RigidBodyDynamics.Mechanism)
        new{T}(mechanism)
    end
end

#TODO: Change Path
Arthur(; mechanism=RigidBodyDynamics.URDF.parse_urdf("../../../kortex_description/arms/gen3/7dof/urdf/GEN3_URDF_V12.urdf", remove_fixed_tree_joints = false)) = Arthur(mechanism)

Arthur

In [22]:
# State, s, is [q q̇ x ẋ F]
# x will be input from the camera
# q, q̇, ẋ will be taken or derived from the arm
# F will be input from the F/T Sensor
# Input, u, is Torque (τ)
function RobotDynamics.dynamics(model::Arthur, x, u)
    # Create a state of the mechanism model and a result struct for the dynamics
    dynamicsResult = RigidBodyDynamics.DynamicsResult(model.mechanism)
    mechanismState = RigidBodyDynamics.MechanismState(model.mechanism)
    
    # Get states and constants of system not dependent on model state
    M = RigidBodyDynamics.mass_matrix(mechanismState)
    num_q = RigidBodyDynamics.num_positions(model.mechanism)
    q = x[1:num_q]
    q̇ = x[num_q+1:2*num_q]
    p = x[2*num_q + 1:2*num_q + 6]
    ṗ = x[2*num_q + 7:2*num_q + 12]
    F = x[2*num_q + 13:2*num_q + 18]
    Be = zeros(6, 6)
    if (norm(ṗ) > 1e-5)
        for k = 1:6
            Be[k,k] = norm(F) / norm(ṗ)
        end
    end
    
    # Set mechanism state to current state
    RigidBodyDynamics.set_configuration!(mechanismState, q)
    RigidBodyDynamics.set_velocity!(mechanismState, q̇)
    
    w = Wrench(default_frame(bodies(model.mechanism)[end-1]), F[4:6], F[1:3])
    wrenches = BodyDict{Wrench{Float64}}(b => zero(Wrench{Float64}, root_frame(model.mechanism)) for b in bodies(model.mechanism))
    wrenches[bodies(model.mechanism)[end-1].id] = transform(w, transform_to_root(mechanismState, bodies(model.mechanism)[end-1]))
    dynamics!(dynamicsResult, mechanismState, u, wrenches)

    q̈ = dynamicsResult.v̇
    p̈ = [dynamicsResult.accelerations[bodies(model.mechanism)[end].id].linear; dynamicsResult.accelerations[bodies(model.mechanism)[end].id].angular]
    Ḟ = Be*p̈
    return SVector{32}([q̇; q̈; ṗ; p̈; Ḟ])
end

RobotDynamics.state_dim(::Arthur) = 32
RobotDynamics.control_dim(::Arthur) = 7

In [23]:
model = Arthur()
# print((RobotDynamics.dynamics(model, zeros(32), 0.1*ones(7))))

Arthur{Any}(Spanning tree:
Vertex: world (root)
  Vertex: base_link, Edge: base_link_to_world
    Vertex: Shoulder_Link, Edge: Actuator1
      Vertex: HalfArm1_Link, Edge: Actuator2
        Vertex: HalfArm2_Link, Edge: Actuator3
          Vertex: ForeArm_Link, Edge: Actuator4
            Vertex: SphericalWrist1_Link, Edge: Actuator5
              Vertex: SphericalWrist2_Link, Edge: Actuator6
                Vertex: Bracelet_Link, Edge: Actuator7
                  Vertex: EndEffector_Link, Edge: EndEffector
No non-tree joints.)

In [75]:
model = Arthur()
mechanismState = RigidBodyDynamics.MechanismState(model.mechanism)
rootframe = root_frame(model.mechanism)
# print(typeof(transform_to_root(mechanismState, bodies(model.mechanism)[end])))
# print("\n\n")
dynamicsResult = RigidBodyDynamics.DynamicsResult(model.mechanism)
# print(dynamicsResult)
# print("\n\n")
RigidBodyDynamics.set_configuration!(mechanismState, rand(Float64, 7))
RigidBodyDynamics.set_velocity!(mechanismState, rand(Float64, 7))
wrenches = BodyDict{Wrench{Float64}}(b => zero(Wrench{Float64}, rootframe) for b in bodies(model.mechanism))
angular = [0, 0.1, 0.2]
linear = [2, 1, 16]
w = Wrench(default_frame(bodies(model.mechanism)[end]), angular, linear)
# print(transform(w, transform_to_root(mechanismState, bodies(model.mechanism)[end])))
# print("\n\n")
wrenches[bodies(model.mechanism)[end].id] = transform(w, transform_to_root(mechanismState, bodies(model.mechanism)[end]))
# print(wrenches)
u = zeros(7)
u[7] = 10
dynamics!(dynamicsResult, mechanismState, u, wrenches)
print(mechanismState.v)
print("\n\n")
print([dynamicsResult.accelerations[bodies(model.mechanism)[end].id].linear; dynamicsResult.accelerations[bodies(model.mechanism)[end].id].angular])
# J1 = getJacobian(model, 0.5*ones(7), 0.2*ones(7))
# J2 = getJacobian(model, 0.5*ones(7), 0.1*ones(7))
# print(J1-J2)
# print("\n\n")
# ẋ1 = getẊ(model, 0.5*ones(7), 0.2*ones(7))
# ẋ2 = getẊ(model, 0.5*ones(7), 0.2*ones(7))
# print(ẋ1-ẋ2)
# print("\n\n")
# print(typeof(getẊ(model, J, zeros(7), 0.1*ones(7))))
# print(getJ̇(model, 0.5*ones(7), 0.2*ones(7)))
# print(typeof(RobotDynamics.dynamics(model, zeros(32), 0.1*ones(7))))

[0.8696004300432425, 0.7057441188247451, 0.08746677084460064, 0.7077920045486048, 0.1578896665213998, 0.06399217179072769, 0.16020516457111156]

[0.5181680395922416, 0.013534196412353927, 9.986683136337955, 0.4621751973861481, -0.7266713791553054, -0.009713552254769608]

In [ ]:
# # State, s, is [q q̇ x ẋ F]
# # x will be input from the camera
# # q, q̇, ẋ will be taken or derived from the arm
# # F will be input from the F/T Sensor
# # Input, u, is Torque (τ)
# function RobotDynamics.dynamics(model::Arthur, s, u)
#     # Create a state of the mechanism model and a result struct for the dynamics
#     dynamicsResult = RigidBodyDynamics.DynamicsResult(model.mechanism)
#     mechanismState = RigidBodyDynamics.MechanismState(model.mechanism)
    
#     # Get states and constants of system not dependent on model state
#     M = RigidBodyDynamics.mass_matrix(mechanismState)
#     num_q = RigidBodyDynamics.num_positions(model.mechanism)
#     q = s[1:num_q]
#     q̇ = s[num_q+1:2*num_q]
#     x = s[2*num_q + 1:2*num_q + 6]
#     ẋ = s[2*num_q + 7:2*num_q + 12]
#     F = s[2*num_q + 13:2*num_q + 18]
#     Be = zeros(6, 6)
#     if (norm(ẋ) > 1e-5)
#         for k = 1:3
#             Be[k,k] = norm(F) / norm(ẋ)
#         end
#     end
    
#     # Set mechanism state to current state
#     RigidBodyDynamics.set_configuration!(mechanismState, q)
#     RigidBodyDynamics.set_velocity!(mechanismState, q̇)
    
#     # Get variables dependent on state
#     J = getJacobian(model, q, q̇)
#     τ_ext = transpose(J)*F
    
#     # Calculate dynamics
#     RigidBodyDynamics.dynamics!(dynamicsResult, mechanismState, u)
#     # Add the effects of external forces/torques into dynamics
#     q̈ = M\((M * dynamicsResult.v̇) - τ_ext)
#     ẍ = getJ̇(model, q, q̇)*q̇ + J*q̈
#     Ḟ = Be*ẍ
#     return [q̇; q̈; ẋ; ẍ; Ḟ]
# end

# function getJacobian(model::Arthur, q,  q̇)
#     mechanismState = RigidBodyDynamics.MechanismState(model.mechanism)
#     RigidBodyDynamics.set_configuration!(mechanismState, q)
#     RigidBodyDynamics.set_velocity!(mechanismState, q̇)
#     p = RigidBodyDynamics.path(model.mechanism, RigidBodyDynamics.root_body(model.mechanism), RigidBodyDynamics.bodies(model.mechanism)[end])
#     J_data = RigidBodyDynamics.geometric_jacobian(mechanismState, p)
# #     print(typeof([J_data.linear; J_data.angular]))
#     return [J_data.linear; J_data.angular]
# end

# function getẊ(model::Arthur, q, q̇)
#     J = getJacobian(model, q, q̇)
#     ẋ = J*q̇
# #     print(typeof(ẋ))
#     return ẋ
# end

# function getJ̇(model::Arthur, q, q̇)
#     return ForwardDiff.jacobian(dq -> getẊ(model, dq, q̇), q)
# end